In [1]:
# import libraries
import os # file I/O
import numpy as np # scientific computing 
import pandas as pd # data analysis
import cv2 # computer vision
import gc # garbage collection 
from tqdm import tqdm # create for loops
from glob import glob # define tha path of all images

## Step-1 and 2
- Collect all data
- Labeling

In [2]:
dirs = os.listdir('data')
images_path = []
labels = []
for folder in dirs:
    path = glob('./data/{}/*.jpg'.format(folder))
    label =['{}'.format(folder)]*len(path)
    # append
    images_path.extend(path)
    labels.extend(label)

## Step-3
- Face detection

In [3]:
img_path = images_path[1]
img = cv2.imread(img_path) # read the image via openCV

In [4]:
# cv2.imshow('original',img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [5]:
# face detection
# load face detection model
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [6]:
def face_detection_dnn(img):
    # blob from image (rgb mean subraction image)
    image = img.copy()
    # height and width
    h,w = image.shape[:2] 
    # calculate the blob from image using CV2 and DNN
    blob = cv2.dnn.blobFromImage(image, 1, (300, 300), (104, 117, 123), swapRB=True)
    # get the detections
    face_detection_model.setInput(blob)
    detections = face_detection_model.forward()
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2] # confidence score
        if confidence > 0.5:
                # indexes 3- 7 contain bonunding box info. Normalize by multipling to the height and width array
                box = detections[0,0,i,3:7]*np.array([w,h,w,h]) 
                box = box.astype(int)
                #print(box) pt1 & pt2 are the 2 diaganol points of the rectangle
                pt1 = (box[0],box[1])
                pt2 = (box[2],box[3])
                #cv2.rectangle(image,pt1,pt2,(0,255,0),2) # thickness of the bounding box is 2 pixels
                roi = image[box[1]:box[3],box[0]:box[2]] # region of interest to get cropped face

                return roi     
    return None

In [7]:
img_roi = face_detection_dnn(img)

In [8]:
# cv2.imshow('roi',img_roi)
# cv2.imshow('original',img)
# cv2.waitKey()
# cv2.destroyAllWindows()

## Step 5
- Blob from image

In [9]:
def datapreprocess(img):
    # blob from image (rgb mean subtraction image)
    face = face_detection_dnn(img)
    if face is not None:

        # computing blob from image. (224, 224) is the size of image, (104,117,123) is the mean RGB value
        # create a 4-D blob from image
        blob = cv2.dnn.blobFromImage(face,1,(224,224),(104,117,123),swapRB=True)
        
        # reduce dimension of image to 3_D. Transpose the 3-D
        blob_squeeze = np.squeeze(blob).T
        
        # rotate the image 90 degrees clockwise
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        
        # mirror image (flip)
        blob_flip = cv2.flip(blob_rotate,1)
        
        # remove negative values and normalize
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()
    
        return img_norm
    else:
        return None

### Apply to all images and create a list

In [10]:
#len(images_path)

data_img = []
label_img = []
i = 0

# loop path and label at the same time
for path, label in tqdm(zip(images_path,labels),desc='preprocessing'):
    img = cv2.imread(path)
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        label_img.append(label)
          
    i += 1
    if i%100 == 0:
        gc.collect() # cleaar temperary MEM every 100 iterations

preprocessing: 10000it [05:10, 32.20it/s]


In [11]:
X = np.array(data_img)
y = np.array(label_img)

In [12]:
# number of image and the dimensions (100, 100, 3). less than 10000, might be due to faca ont identifiable
X.shape, y.shape 

((9959, 224, 224, 3), (9959,))

In [13]:
np.savez('./data/data_preprocess.npz',X,y)